In [43]:
import json, copy, re
import pandas as pd

In [44]:
with open("uca.json", 'r') as file:
    data = json.load(file)
    
profiles = copy.deepcopy(data)
articles_data = copy.deepcopy(data)

# Transform profile Data

#### Transform the `Article` and `Spciality`

In [45]:
for item in profiles:
    item["Spciality"] = ", ".join(item["Spciality"])
    item["Article"] = len(item["Article"])

#### Add 'Faculty'

In [46]:
class Transform:
    def __init__(self, to, test) -> None:
        self.value = to
        self.test = test


def test(e, search):
    return any(s.lower() in e.lower() for s in search)


def transform_to_fstg(e):
    return test(e, ["FST", "FSTG", "Faculty of Sciences and Techniques", 
                    "Faculté des Sciences et Techniques", "Faculty of Science and Technology", 
                    "Computer Science", "Computer", "Biologie","informatique", "Géographie",
                    "Biology & Environmental Engineering", "Environmental Engineering", "chimie"
                    ])

def transform_to_ensa(e):
    return test(e, ["ENSA"])

def transform_to_fssm(e):
    return test(e, ["FSSM","Mathématiques", "Physics","physique", "Semlalia", 
                    "Faculty of Sciences", "Hydrogéologie", "Hydrologie", "physico-chimie",
                    "Géologie", "mathématique", "microbiologie","mathématiques"
                    ])

def transform_to_fmpm(e):
    return test(e, ["FMPm", "Faculté de Médecine et de Pharmacie", "Faculté de Médecine"
                    "Médecine", "Pharmacie", "médicale", "Faculty of medecine", "Algologie",
                    "anesthésie réanimation", "réanimation", "anesthésie", "Hopital",
                    "Toxicology and Food Safety"
                    ])

def transform_to_fless(e):
    return test(e, ["Faculty of Law, Economic and Social Sciences", "Faculty of Law",
                    "Economic", "Social Sciences", "Social"])

def transform_to_este(e):
    return test(e, ["supérieure de Technologie d'Essaouira",
                    "supérieure de Technologie d'Essaouira",
                    "Superior School of Technology KS",
                    "Superior School of Technology",
                    "School of Technology", "Superior School of Technology",
                    "ESTE"])

def transform_to_fps(e):
    return test(e, ["FPS", "FPS - Safi", "Faculté Polydisciplinaire de Safi", "Polydisciplinaire"
                    "Faculté Polydisciplinaire", "Faculté Poly disciplinaire", "Poly disciplinaire"
                    ])

def transform_to_other(e):
    # return e == "Université Cadi Ayyad" or e == "CADI AYYAD UNIVERSITY"
    return test(e, ["Université Cadi Ayyad",
                    "Cadi Ayyad University",
                    "Cadi  Ayyad",
                    "CADI AYYAD UNIVERSITY"
                    ])

faculties_transform = [
    Transform("FSTG", transform_to_fstg),
    Transform("ENSA", transform_to_ensa),
    Transform("FSSM", transform_to_fssm),
    Transform("FPS", transform_to_fps),
    Transform("FMPM", transform_to_fmpm),
    Transform("FLESS", transform_to_fless),
    Transform("FPS", transform_to_fps),
    Transform("ESTE", transform_to_este),
    Transform("Other", transform_to_other),
]


def transform_faculties(data):
    for ind, item in enumerate(data):
        for transform in faculties_transform:
            if transform.test(item.get("ASS - Unevercity", "")):
                data[ind]["faculty"] = transform.value
                break
            #else: data[ind]["faculty"] = item.get("ASS - Unevercity")
            else: data[ind]["faculty"] = "None"

In [47]:
# from faculty_transform import transform_faculties
profiles = copy.deepcopy(profiles)
transform_faculties(profiles)

In [48]:
for prof in profiles:
    print(prof["faculty"])

In [49]:
profiles_df = pd.DataFrame(profiles)
profiles_df

In [50]:
profiles_df.to_csv("./generated/profiles-v2.csv", index=False)

In [51]:
with open("./generated/faculty_specialty.json", 'w') as file:
    json.dump(profiles, file, indent=4)